In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(codeList):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2978, 거래소 1562, 코스닥 1416
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (1562 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 280,579,754,000,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 57,948,988,254,000
삼성전자우 상장주식수: 822,886,700, 시가총액 32,750,890,660,000
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 30,932,137,500,000
NAVER 상장주식수: 164,263,395, 시가총액 28,253,303,940,000
셀트리온 상장주식수: 128,346,619, 시가총액 25,155,937,324,000
LG화학 상장주식수: 70,592,343, 시가총액 20,507,075,641,500
현대차 상장주식수: 213,668,187, 시가총액 18,439,564,538,100
LG생활건강 상장주식수: 15,618,197, 시가총액 18,179,581,308,000
삼성물산 상장주식수: 189,690,043, 시가총액 16,085,715,646,400
삼성SDI 상장주식수: 68,764,530, 시가총액 15,918,988,695,000
현대모비스 상장주식수: 95,054,694, 시가총액 15,684,024,510,000
SK텔레콤 상장주식수: 80,745,711, 시가총액 14,776,465,113,000
엔씨소프트

한전기술 상장주식수: 38,220,000, 시가총액 577,122,000,000
미래에셋생명 상장주식수: 177,016,189, 시가총액 566,451,804,800
후성 상장주식수: 92,606,819, 시가총액 565,827,664,090
쿠쿠홀딩스 상장주식수: 7,112,437, 시가총액 547,657,649,000
종근당홀딩스 상장주식수: 5,009,861, 시가총액 543,569,918,500
동원시스템즈 상장주식수: 25,618,511, 시가총액 537,988,731,000
SPC삼립 상장주식수: 8,629,009, 시가총액 534,998,558,000
KINDEX 200 상장주식수: 22,100,000, 시가총액 524,101,500,000
하나투어 상장주식수: 13,939,185, 시가총액 522,022,478,250
빙그레 상장주식수: 9,851,241, 시가총액 516,205,028,400
동아쏘시오홀딩스 상장주식수: 6,138,297, 시가총액 513,161,629,200
풍산 상장주식수: 28,024,278, 시가총액 511,443,073,500
보령제약 상장주식수: 44,200,000, 시가총액 508,300,000,000
씨에스윈드 상장주식수: 17,285,715, 시가총액 507,335,735,250
KODEX WTI원유선물(H) 상장주식수: 61,650,000, 시가총액 501,522,750,000
한섬 상장주식수: 24,630,000, 시가총액 501,220,500,000
HDC 상장주식수: 59,741,721, 시가총액 494,661,449,880
락앤락 상장주식수: 54,186,600, 시가총액 493,639,926,000
SK디앤디 상장주식수: 18,990,164, 시가총액 491,845,247,600
제이콘텐트리 상장주식수: 16,385,071, 시가총액 489,094,369,350
KOSEF 200 상장주식수: 20,250,000, 시가총액 482,861,250,000
넥센타이어 상장주식수: 97,667,877, 시가총액

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [5]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [6]:
import pandas as pd

In [7]:
format(100000000000, ",")

'100,000,000,000'

In [8]:
format(1000000000000, ",")

'1,000,000,000,000'

In [9]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [10]:
# pdMarketTotal.to_csv("../data/KOSPI_LIST.csv", index=None)

In [12]:
# pdMarketTotal = pdMarketTotal.loc[(
    
#     (pdMarketTotal[0].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal[0].apply(lambd`a x: x.find("SMART") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal[0].apply(lambda x: x.find("ETF") < 0)) 
    
# )]

In [28]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization".split(",")

In [29]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization
105,KODEX 200선물인버스2X,A252670,178800000,1489404000000


In [16]:
pdMarketTotal_200 = pdMarketTotal[:200]

In [17]:
pdMarketTotal_200.columns = "Name,Code,Stock,Capitalization".split(",")

In [18]:
pdMarketTotal_200.to_csv("../data/KOSPI200_LIST.csv", index=None)

In [40]:
# old = pd.read_csv("../data/KOSPI200_LIST.csv")

# KODEX = pdMarketTotal.loc[(pdMarketTotal.Name.apply(lambda x: "KODEX" in x))]

# out = pdMarketTotal.loc[(
#     (pdMarketTotal.Name.isin(old.Name)) |
#     (pdMarketTotal.Name.isin(KODEX.Name)) |
#     (pdMarketTotal.Name.isin(pdMarketTotal[:100].Name))
# )]

# out.to_csv("../data/KOSPI200_LIST.csv", index=None)